In [1]:
import os
os.chdir('/home/blueconet/Downloads/Image Classification')

In [2]:
from cifar_input import cifar10_input
import ResNet_cifar10_1 as model
import tensorflow as tf

BATCH_SIZE=16
EVAL_SIZE=1000
NUM_GPUS=8

def tower_loss(batch, scope):
    images, labels = batch
    
    logits = model.inference(images, keep_prob)
    _ = model.loss(logits, labels)
    
    losses = tf.get_collection('losses', scope)
    
    total_loss = tf.add_n(losses, name='total_loss')
    return total_loss

def average_grads(list_grads):
    grads = [(tf.reduce_mean([list_grads[j][i][0] for j in range(len(list_grads))],
                             0),
             list_grads[0][i][1]) for i in range(len(list_grads[0]))]
    return grads

In [3]:
with tf.Graph().as_default(), tf.device("/cpu:0"):
    train_data, test_data = cifar10_input()
    
    train_batch = train_data.dist_batch(BATCH_SIZE,20000)
    test_batch = test_data.batch(200)

    x = tf.placeholder(tf.float32, [None, 32, 32, 3])
    y_ = tf.placeholder(tf.int32, [x.get_shape()[0]])
    keep_prob = tf.placeholder(tf.float32)
    lr = tf.placeholder(tf.float32)
    opt = tf.train.GradientDescentOptimizer(lr)

    with tf.device('/gpu:0'):
        logit = model.inference(x, keep_prob)
        loss = model.loss(logit, y_)
        grads = opt.compute_gradients(loss)    
    
    train_op = opt.apply_gradients(grads)
    sess = tf.Session()
    
    rate = 0.001
    
    sess.run(tf.initialize_all_variables())
    tf.train.start_queue_runners(sess=sess)
    for i in range(20):
        image_batch, label_batch = sess.run(train_batch)
        _, cross_entropy = sess.run([train_op, loss], feed_dict = {x: image_batch, y_: label_batch, keep_prob: 0.5, lr:rate})
        print("step %d, cross_entropy %g"%(i, cross_entropy))

step 0, cross_entropy 5.87338
step 1, cross_entropy 3.29626
step 2, cross_entropy 3.09067
step 3, cross_entropy 4.62663
step 4, cross_entropy 2.79843
step 5, cross_entropy 2.3198
step 6, cross_entropy 2.96101
step 7, cross_entropy 2.21669
step 8, cross_entropy 2.57685
step 9, cross_entropy 2.28373
step 10, cross_entropy 2.75828
step 11, cross_entropy 2.631
step 12, cross_entropy 2.42016
step 13, cross_entropy 2.38508
step 14, cross_entropy 2.63291
step 15, cross_entropy 2.51606
step 16, cross_entropy 2.31617
step 17, cross_entropy 2.50716
step 18, cross_entropy 2.11757
step 19, cross_entropy 2.75414


In [4]:
with tf.Graph().as_default(), tf.device("/cpu:0"):
        train_data, test_data = cifar10_input()
        
        train_batch = train_data.dist_batch(BATCH_SIZE, 20000)
        images, labels = test_data.dist_batch(EVAL_SIZE, 4000)

        keep_prob = tf.placeholder(tf.float32)
        lr = tf.placeholder(tf.float32)

        logit = model.inference(images, keep_prob)
        accuracy = tf.reduce_mean(tf.cast(tf.nn.in_top_k(logit, labels, 1),tf.float32))
        tf.get_variable_scope().reuse_variables()
        
        opt = tf.train.GradientDescentOptimizer(lr)

        with tf.device('/gpu:0'):
            with tf.name_scope('%s_%d' % ('tower', 0)) as scope:
                loss = tower_loss(train_batch, scope)
                grads = opt.compute_gradients(loss)    
    
        train_op = opt.apply_gradients(grads)
        sess = tf.Session()
        
        rate = 0.1
        mean_loss = 0.
        least = 10.
        count = 0.
    
        sess.run(tf.initialize_all_variables())
        tf.train.start_queue_runners(sess=sess)
        for i in range(20000):
            _, cross_entropy = sess.run([train_op, loss], feed_dict = {keep_prob: 0.5,
                                                                       lr:rate})
            mean_loss = mean_loss + cross_entropy/50
    
            if i%50 == 0 and i>0:
                print("step %d, cross_entropy %g"%(i, mean_loss))
      
                if mean_loss > least:
                    count = count+1
                    if count > 4:
                        rate = rate/2.0
                        count = 0
                    else:
                        count = 0
                        least = mean_loss
                mean_loss=0
        
            if i%250 == 0 and i>0:
                eval_accuracy = sess.run(accuracy)
                print("test accuracy %g"%(eval_accuracy))

step 50, cross_entropy nan
step 100, cross_entropy nan
step 150, cross_entropy nan
step 200, cross_entropy nan
step 250, cross_entropy nan
test accuracy 0
step 300, cross_entropy nan


KeyboardInterrupt: 

In [4]:
with tf.Graph().as_default(), tf.device("/cpu:0"):
        train_data, test_data = cifar10_input()
        
        train_batch = train_data.dist_batch(BATCH_SIZE, 20000)
        images, labels = test_data.dist_batch(EVAL_SIZE, 4000)

        keep_prob = tf.placeholder(tf.float32)
        lr = tf.placeholder(tf.float32)

        logit = model.inference(images, keep_prob)
        accuracy = tf.reduce_mean(tf.cast(tf.nn.in_top_k(logit, labels, 1),tf.float32))
        tf.get_variable_scope().reuse_variables()
        
        opt = tf.train.GradientDescentOptimizer(lr)
        
        tower_grads = []
        tower_losses = []
        
        for i in range(NUM_GPUS):
            with tf.device('/gpu:%d' % i):
                with tf.name_scope('%s_%d' % ('tower', i)) as scope:
                    loss = tower_loss(train_batch, scope)
                    grads = opt.compute_gradients(loss)
                    tower_losses.append(loss)
                    tower_grads.append(grads)
                    
        grads = average_grads(tower_grads)
        mean_loss = tf.reduce_mean(tower_losses)
  
        train_op = opt.apply_gradients(grads)
        sess = tf.Session()
        
        rate = 0.001
        least = 10.
        mean = 0.
        count = 0.
        highest = 0.
    
        sess.run(tf.initialize_all_variables())
        tf.train.start_queue_runners(sess=sess)
        for i in range(20000):
            _, cross_entropy = sess.run([train_op, mean_loss], feed_dict = {keep_prob: 0.5,
                                                                       lr:rate})
            mean = mean + cross_entropy/200
            if (i+1)%200 == 0 and i>0:
                print("step %d, cross_entropy %g"%(i, cross_entropy))
      
                if mean > least:
                    count = count+1
                    if count > 4:
                        rate = rate/2.0
                        count = 0
                    else:
                        count = 0
                        least = mean
                
                mean=0.
        
            if (i+1)%200 == 0 and i>0:
                eval_accuracy = sess.run(accuracy)
                if eval_accuracy > highest:
                    highest = eval_accuracy
                print("test accuracy %g"%(eval_accuracy))
                
        print("highest accuracy: %g"%(highest))

step 199, cross_entropy 2.04319
test accuracy 0.174
step 399, cross_entropy 1.99578
test accuracy 0.25
step 599, cross_entropy 2.30301
test accuracy 0.271
step 799, cross_entropy 2.03042
test accuracy 0.289
step 999, cross_entropy 1.65223
test accuracy 0.282
step 1199, cross_entropy 1.77055
test accuracy 0.357
step 1399, cross_entropy 1.53573
test accuracy 0.297
step 1599, cross_entropy 1.8673
test accuracy 0.334
step 1799, cross_entropy 2.04246
test accuracy 0.35
step 1999, cross_entropy 1.96681
test accuracy 0.322
step 2199, cross_entropy 1.97293
test accuracy 0.364
step 2399, cross_entropy 1.55557
test accuracy 0.364
step 2599, cross_entropy 1.83661
test accuracy 0.384
step 2799, cross_entropy 1.64741
test accuracy 0.348
step 2999, cross_entropy 1.40832
test accuracy 0.394
step 3199, cross_entropy 1.81047
test accuracy 0.392
step 3399, cross_entropy 2.00359
test accuracy 0.389
step 3599, cross_entropy 1.78116
test accuracy 0.42
step 3799, cross_entropy 1.66743
test accuracy 0.398
st

In [5]:
rate = 0.002
with tf.device('/cpu:0'):
    for i in range(10000):
            _, cross_entropy = sess.run([train_op, mean_loss], feed_dict = {keep_prob: 0.5,
                                                                       lr:rate})
            mean = mean + cross_entropy/200
            if (i+1)%200 == 0 and i>0:
                print("step %d, cross_entropy %g"%(i, cross_entropy))          
                mean=0.
        
            if (i+1)%200 == 0 and i>0:
                eval_accuracy = sess.run(accuracy)
                if eval_accuracy > highest:
                    highest = eval_accuracy
                print("test accuracy %g"%(eval_accuracy))
                
    print("highest accuracy: %g"%(highest))

step 199, cross_entropy 1.52218
test accuracy 0.548
step 399, cross_entropy 1.39162
test accuracy 0.562
step 599, cross_entropy 1.19618
test accuracy 0.563
step 799, cross_entropy 1.08101
test accuracy 0.601
step 999, cross_entropy 1.04709
test accuracy 0.477
step 1199, cross_entropy 1.60989
test accuracy 0.525
step 1399, cross_entropy 0.942184
test accuracy 0.609
step 1599, cross_entropy 1.45045
test accuracy 0.584
step 1799, cross_entropy 0.767681
test accuracy 0.58
step 1999, cross_entropy 1.34619
test accuracy 0.579
step 2199, cross_entropy 1.33081
test accuracy 0.597
step 2399, cross_entropy 0.942777
test accuracy 0.605
step 2599, cross_entropy 1.39814
test accuracy 0.604
step 2799, cross_entropy 1.49116
test accuracy 0.621
step 2999, cross_entropy 1.09268
test accuracy 0.612
step 3199, cross_entropy 0.94688
test accuracy 0.598
step 3399, cross_entropy 0.847004
test accuracy 0.619
step 3599, cross_entropy 1.11168
test accuracy 0.546
step 3799, cross_entropy 1.15331
test accuracy 0

In [10]:
rate = 0.000004
with tf.device('/cpu:0'):
    for i in range(2000):
            _, cross_entropy = sess.run([train_op, mean_loss], feed_dict = {keep_prob: 0.5,
                                                                       lr:rate})
            mean = mean + cross_entropy/200
            if (i+1)%200 == 0 and i>0:
                print("step %d, cross_entropy %g"%(i, cross_entropy))          
                mean=0.
        
            if (i+1)%200 == 0 and i>0:
                eval_accuracy = sess.run(accuracy)
                if eval_accuracy > highest:
                    highest = eval_accuracy
                print("test accuracy %g"%(eval_accuracy))
                
    print("highest accuracy: %g"%(highest))

step 199, cross_entropy 0.520421
test accuracy 0.729
step 399, cross_entropy 0.487627
test accuracy 0.726
step 599, cross_entropy 0.511609
test accuracy 0.71
step 799, cross_entropy 1.13343
test accuracy 0.737
step 999, cross_entropy 1.06184
test accuracy 0.729
step 1199, cross_entropy 0.560799
test accuracy 0.732
step 1399, cross_entropy 0.491114
test accuracy 0.742
step 1599, cross_entropy 1.05102
test accuracy 0.719
step 1799, cross_entropy 0.656647
test accuracy 0.747
step 1999, cross_entropy 0.76131
test accuracy 0.745
highest accuracy: 0.749
